In [8]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [9]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [10]:
subject = 'Acre - Consumo de Cimento (t)'
split_index = 203 #Referente aos 230 anos de input  
train_split = split_index + 1 - 12*3

In [11]:
data = pd.read_csv('2003_mo_model_input_AC.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,8.722906,0.691320,7.285707e+06,413409.255032,9.318417,6.587220e+06,3.260
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,8.718028,0.691617,7.294881e+06,413607.843560,9.319990,6.590338e+06,2.191
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,8.713149,0.691914,7.304055e+06,413806.432089,9.321563,6.593455e+06,2.154
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,8.708271,0.692212,7.313229e+06,414005.020618,9.323136,6.596572e+06,2.643
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,8.703393,0.692509,7.322403e+06,414203.609146,9.324709,6.599689e+06,2.546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.748
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.032
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.573
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.493


In [12]:
input_data = data.iloc[:split_index + 1,1:-1]
# input_data = data.iloc[:split_index + 1,:]
input_data = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado
0,2.782450,4.506880,2.067266,2.574314,-2.064648,-2.469876,3.184489,-0.871260,-2.233199,-1.827644,0.550935,-0.265790,-2.404401
1,2.407943,4.328460,1.285816,2.334870,-2.037913,-2.431875,3.029073,-0.874207,-2.196498,-1.807633,0.567407,-0.242610,-2.361200
2,2.179073,4.129086,2.919965,2.221334,-2.011179,-2.393874,2.926505,-0.877154,-2.159796,-1.787622,0.583879,-0.219431,-2.317998
3,2.077086,3.911409,1.142823,2.040542,-1.984445,-2.355872,2.828220,-0.880101,-2.123094,-1.767611,0.600352,-0.196252,-2.274796
4,1.942128,3.663912,2.355956,1.912744,-1.957710,-2.317871,3.036493,-0.883048,-2.086392,-1.747600,0.616824,-0.173072,-2.231595
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,-0.689886,-0.724085,-1.189161,3.148408,1.378950,0.117681,0.370628,1.086583,1.093639,0.845955,-0.984945,-1.492171,0.341015
200,-0.441954,-0.736434,-1.296499,3.219670,1.391539,0.092456,0.427775,1.079514,1.075237,0.829861,-0.967308,-1.480041,0.326774
201,-0.132782,-0.738433,-1.444029,3.421082,1.404128,0.067231,0.538287,1.072445,1.056835,0.813767,-0.949672,-1.467910,0.312533
202,0.084061,-0.738236,-1.444143,3.373840,1.416717,0.042006,0.666156,1.065376,1.038433,0.797673,-0.932035,-1.455780,0.298292


In [13]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0       2.191
1       2.154
2       2.643
3       2.546
4       3.040
        ...  
235    20.032
236    18.573
237    15.493
238    15.493
239       NaN
Name: Acre - Consumo de Cimento (t), Length: 240, dtype: float64

In [14]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado
0,2.782450,4.506880,2.067266,2.574314,-2.064648,-2.469876,3.184489,-0.871260,-2.233199,-1.827644,0.550935,-0.265790,-2.404401
1,2.407943,4.328460,1.285816,2.334870,-2.037913,-2.431875,3.029073,-0.874207,-2.196498,-1.807633,0.567407,-0.242610,-2.361200
2,2.179073,4.129086,2.919965,2.221334,-2.011179,-2.393874,2.926505,-0.877154,-2.159796,-1.787622,0.583879,-0.219431,-2.317998
3,2.077086,3.911409,1.142823,2.040542,-1.984445,-2.355872,2.828220,-0.880101,-2.123094,-1.767611,0.600352,-0.196252,-2.274796
4,1.942128,3.663912,2.355956,1.912744,-1.957710,-2.317871,3.036493,-0.883048,-2.086392,-1.747600,0.616824,-0.173072,-2.231595
...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,-0.742853,-0.619453,0.617009,-0.970725,0.980624,0.764972,-1.322806,1.310355,1.290080,1.098918,-1.469526,-1.416170,0.668474
164,-0.787367,-0.611176,0.459632,-0.841670,0.991608,0.753473,-1.355523,1.299668,1.287297,1.100953,-1.469505,-1.430567,0.666551
165,-0.757940,-0.599627,0.493659,-0.838453,1.002592,0.741973,-1.376175,1.288982,1.284515,1.102988,-1.469485,-1.444964,0.664628
166,-0.917469,-0.589237,0.617024,-1.006001,1.013577,0.730473,-1.414146,1.278295,1.281733,1.105023,-1.469464,-1.459361,0.662705


In [15]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0       2.191000
1       2.154000
2       2.643000
3       2.546000
4       3.040000
         ...    
163    10.885543
164    10.765657
165    10.645771
166    10.525886
167    10.406000
Name: Acre - Consumo de Cimento (t), Length: 168, dtype: float64

In [16]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado
168,-1.032458,-0.585256,-0.505289,-1.202612,1.035545,0.707473,-1.446117,1.256922,1.276169,1.109094,-1.469423,-1.488155,0.658858
169,-1.077854,-0.581056,-0.373612,-1.139987,1.046449,0.692399,-1.526274,1.252043,1.283418,1.106798,-1.455041,-1.495061,0.654927
170,-1.036724,-0.574769,-0.291560,-1.053534,1.057352,0.677324,-1.610341,1.247165,1.290668,1.104502,-1.440660,-1.501968,0.650996
171,-1.013400,-0.567021,-0.450118,-0.932974,1.068256,0.662249,-1.601301,1.242287,1.297918,1.102206,-1.426278,-1.508874,0.647064
172,-1.059228,-0.557842,-0.909054,-0.763756,1.079159,0.647175,-1.618284,1.237408,1.305168,1.099910,-1.411896,-1.515781,0.643133
173,-1.170032,-0.548012,0.387442,-0.644652,1.090063,0.632100,-1.602348,1.232530,1.312418,1.097614,-1.397514,-1.522688,0.639202
174,-1.115621,-0.537244,-0.937589,-0.541601,1.100966,0.617026,-1.686659,1.227652,1.319668,1.095318,-1.383133,-1.529594,0.635270
175,-1.182159,-0.524149,-1.212466,-0.565747,1.111870,0.601951,-1.688184,1.222773,1.326918,1.093022,-1.368751,-1.536501,0.631339
176,-1.270623,-0.519647,-0.499856,-0.536256,1.122773,0.586877,-1.692173,1.217895,1.334168,1.090727,-1.354369,-1.543408,0.627408
177,-1.321931,-0.510389,-0.120028,-0.544036,1.133677,0.571802,-1.690760,1.213016,1.341418,1.088431,-1.339988,-1.550314,0.623476


In [17]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

168     9.081
169     9.545
170    10.130
171    14.451
172    13.218
173    13.552
174    15.826
175    14.869
176    14.734
177    12.754
178    11.925
179    13.114
180     8.920
181     9.724
182     9.737
183    10.136
184    14.403
185    12.871
186    15.749
187    14.269
188    15.367
189    14.595
190     9.811
191    11.188
192    10.381
193     8.080
194    11.154
195    12.508
196    12.126
197    14.496
198    16.723
199    15.253
200    16.531
201    15.206
202    10.160
203    13.188
Name: Acre - Consumo de Cimento (t), dtype: float64

In [37]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*6 + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [38]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=100)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 8)
    target,target_val = validation_splitter(train_target, 8)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [29]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][100:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [30]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3300851068, 2591116789, 3539275307, 747374376, 3464644691, 11710472, 1581377757, 2997616177, 881700099, 3914593484, 735457813, 1893950898, 624092425, 1621762519, 4032288030, 2329170196, 2636346704, 2226565916, 4206481977, 3685193777, 3589539910, 2011451999, 3567936467, 606819754, 1544376240]


Step: 0 ___________________________________________
val_loss: 4.629240989685059
winner_seed: 3300851068


Step: 1 ___________________________________________
val_loss: 3.636111259460449
winner_seed: 2591116789


Step: 2 ___________________________________________
val_loss: 4.585196495056152


Step: 3 ___________________________________________
val_loss: 4.060478687286377


Step: 4 ___________________________________________
val_loss: 4.109124183654785


Step: 5 ___________________________________________
val_loss: 3.6375255584716797


Step: 6 ___________________________________________
val_loss: 3.0314462184906006
winner_seed: 1581377757


Step: 7 ___________________________________________
val_

In [31]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 0s 21ms/step - loss: 84.6371 - val_loss: 121.9403
Epoch 2/10000
5/5 [==============================] - 0s 5ms/step - loss: 36.2487 - val_loss: 5.7643
Epoch 3/10000
5/5 [==============================] - 0s 5ms/step - loss: 7.8692 - val_loss: 6.3992
Epoch 4/10000
5/5 [==============================] - 0s 4ms/step - loss: 7.7729 - val_loss: 6.2975
Epoch 5/10000
5/5 [==============================] - 0s 4ms/step - loss: 7.7427 - val_loss: 5.3499
Epoch 6/10000
5/5 [==============================] - 0s 5ms/step - loss: 7.1702 - val_loss: 6.4911
Epoch 7/10000
5/5 [==============================] - 0s 5ms/step - loss: 8.2551 - val_loss: 7.9489
Epoch 8/10000
5/5 [==============================] - 0s 5ms/step - loss: 6.7186 - val_loss: 7.0192
Epoch 9/10000
5/5 [==============================] - 0s 5ms/step - loss: 6.5129 - val_loss: 5.3453
Epoch 10/10000
5/5 [==============================] - 0s 5ms/step - loss: 7.1810 - val_loss: 10.3465
Epo

Epoch 83/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.6666 - val_loss: 6.4332
Epoch 84/10000
5/5 [==============================] - 0s 5ms/step - loss: 5.4471 - val_loss: 6.4626
Epoch 85/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.4322 - val_loss: 5.4888
Epoch 86/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.3888 - val_loss: 5.4705
Epoch 87/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.3335 - val_loss: 7.5533
Epoch 88/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.5645 - val_loss: 5.6862
Epoch 89/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.5753 - val_loss: 5.5448
Epoch 90/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.1559 - val_loss: 5.8580
Epoch 91/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.1962 - val_loss: 5.6489
Epoch 92/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.4769 - val_loss: 6.2979


5/5 [==============================] - 0s 4ms/step - loss: 5.0453 - val_loss: 6.6831
Epoch 246/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.0811 - val_loss: 6.4953
Epoch 247/10000
5/5 [==============================] - 0s 4ms/step - loss: 4.9492 - val_loss: 5.5172
Epoch 248/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.2053 - val_loss: 5.7792
Epoch 249/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.0365 - val_loss: 6.1428
Epoch 250/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.0010 - val_loss: 5.5269
Epoch 251/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.1744 - val_loss: 5.4815
Epoch 252/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.0887 - val_loss: 5.4645
Epoch 253/10000
5/5 [==============================] - 0s 4ms/step - loss: 5.4373 - val_loss: 5.4746
Epoch 254/10000
5/5 [==============================] - 0s 4ms/step - loss: 4.9486 - val_loss: 6.4144
Epoch 

5/5 [==============================] - 0s 5ms/step - loss: 3.9880 - val_loss: 5.0424
Epoch 408/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.4090 - val_loss: 6.7998
Epoch 409/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.2614 - val_loss: 5.2633
Epoch 410/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.0374 - val_loss: 4.4152
Epoch 411/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.2389 - val_loss: 5.4654
Epoch 412/10000
5/5 [==============================] - 0s 5ms/step - loss: 3.9405 - val_loss: 7.2134
Epoch 413/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.0031 - val_loss: 5.9047
Epoch 414/10000
5/5 [==============================] - 0s 5ms/step - loss: 4.0618 - val_loss: 6.2962
Epoch 415/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.8864 - val_loss: 4.9506
Epoch 416/10000
5/5 [==============================] - 0s 5ms/step - loss: 3.8213 - val_loss: 5.9919
Epoch 

5/5 [==============================] - 0s 4ms/step - loss: 3.3542 - val_loss: 5.3823
Epoch 570/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.9405 - val_loss: 4.9926
Epoch 571/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.9018 - val_loss: 10.8529
Epoch 572/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.3883 - val_loss: 5.6806
Epoch 573/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.4743 - val_loss: 4.7137
Epoch 574/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.9382 - val_loss: 8.4577
Epoch 575/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.5982 - val_loss: 4.5462
Epoch 576/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.4671 - val_loss: 7.3604
Epoch 577/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.1439 - val_loss: 4.6893
Epoch 578/10000
5/5 [==============================] - 0s 5ms/step - loss: 3.6547 - val_loss: 7.6530
Epoch

5/5 [==============================] - 0s 4ms/step - loss: 2.6582 - val_loss: 8.5491
Epoch 732/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.5618 - val_loss: 7.5881
Epoch 733/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.8762 - val_loss: 5.8988
Epoch 734/10000
5/5 [==============================] - 0s 4ms/step - loss: 3.0490 - val_loss: 4.8349
Epoch 735/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.6937 - val_loss: 5.2410
Epoch 736/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.8420 - val_loss: 4.4517
Epoch 737/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.8747 - val_loss: 4.6356
Epoch 738/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.8892 - val_loss: 5.2039
Epoch 739/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.7444 - val_loss: 4.6017
Epoch 740/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.7562 - val_loss: 6.4633
Epoch 

5/5 [==============================] - 0s 4ms/step - loss: 2.4538 - val_loss: 8.2912
Epoch 894/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.4363 - val_loss: 5.1608
Epoch 895/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.5077 - val_loss: 5.1780
Epoch 896/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.7646 - val_loss: 6.7556
Epoch 897/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.6334 - val_loss: 4.6363
Epoch 898/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.2009 - val_loss: 5.4693
Epoch 899/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.3652 - val_loss: 4.9660
Epoch 900/10000
5/5 [==============================] - 0s 5ms/step - loss: 2.3312 - val_loss: 5.2417
Epoch 901/10000
5/5 [==============================] - 0s 5ms/step - loss: 2.5798 - val_loss: 4.7938
Epoch 902/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.2864 - val_loss: 6.8799
Epoch 

5/5 [==============================] - 0s 4ms/step - loss: 2.2492 - val_loss: 5.4211
Epoch 1055/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.8428 - val_loss: 6.1055
Epoch 1056/10000
5/5 [==============================] - 0s 5ms/step - loss: 2.1266 - val_loss: 5.3673
Epoch 1057/10000
5/5 [==============================] - 0s 5ms/step - loss: 2.3739 - val_loss: 5.0079
Epoch 1058/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.1288 - val_loss: 4.7558
Epoch 1059/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.9883 - val_loss: 4.7624
Epoch 1060/10000
5/5 [==============================] - 0s 5ms/step - loss: 2.3467 - val_loss: 5.9667
Epoch 1061/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.1968 - val_loss: 3.9899
Epoch 1062/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.5666 - val_loss: 6.1857
Epoch 1063/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.0336 - val_loss: 4.88

5/5 [==============================] - 0s 5ms/step - loss: 1.7311 - val_loss: 7.0411
Epoch 1215/10000
5/5 [==============================] - 0s 5ms/step - loss: 1.9576 - val_loss: 8.6151
Epoch 1216/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.8704 - val_loss: 4.1546
Epoch 1217/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.1842 - val_loss: 5.9038
Epoch 1218/10000
5/5 [==============================] - 0s 5ms/step - loss: 1.7531 - val_loss: 4.6213
Epoch 1219/10000
5/5 [==============================] - 0s 5ms/step - loss: 1.9054 - val_loss: 5.2823
Epoch 1220/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.7815 - val_loss: 7.9889
Epoch 1221/10000
5/5 [==============================] - 0s 4ms/step - loss: 2.2278 - val_loss: 4.0792
Epoch 1222/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.6040 - val_loss: 6.3101
Epoch 1223/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.8757 - val_loss: 5.42

5/5 [==============================] - 0s 4ms/step - loss: 1.2812 - val_loss: 6.2619
Epoch 1375/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.3258 - val_loss: 4.2195
Epoch 1376/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.2017 - val_loss: 5.4796
Epoch 1377/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.1679 - val_loss: 7.4289
Epoch 1378/10000
5/5 [==============================] - 0s 5ms/step - loss: 1.2555 - val_loss: 5.0506
Epoch 1379/10000
5/5 [==============================] - 0s 5ms/step - loss: 1.2664 - val_loss: 12.0695
Epoch 1380/10000
5/5 [==============================] - 0s 5ms/step - loss: 1.1545 - val_loss: 5.9778
Epoch 1381/10000
5/5 [==============================] - 0s 5ms/step - loss: 1.3019 - val_loss: 4.8551
Epoch 1382/10000
5/5 [==============================] - 0s 7ms/step - loss: 1.5901 - val_loss: 6.1562
Epoch 1383/10000
5/5 [==============================] - 0s 4ms/step - loss: 1.2677 - val_loss: 6.5

In [32]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        error = np.abs(prediction - test_target[start_target + i])
        errors.append(error)
        error_percent.append(error / test_target[start_target + i])
        print(f"Month-{i + 1} - Error: {error}")
    
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [33]:
errors, mae, mape = mae_mape_calculator(trained_model, test_input, test_target, 168)

1/1 [==============================] - 0s 50ms/step
Month-1 - Error: [[0.6973934]]
1/1 [==============================] - 0s 13ms/step
Month-2 - Error: [[0.31512356]]
1/1 [==============================] - 0s 13ms/step
Month-3 - Error: [[0.35006905]]
1/1 [==============================] - 0s 13ms/step
Month-4 - Error: [[5.304761]]
1/1 [==============================] - 0s 14ms/step
Month-5 - Error: [[7.8966503]]
1/1 [==============================] - 0s 14ms/step
Month-6 - Error: [[3.6201324]]
1/1 [==============================] - 0s 13ms/step
Month-7 - Error: [[12.103936]]
1/1 [==============================] - 0s 13ms/step
Month-8 - Error: [[14.504537]]
1/1 [==============================] - 0s 14ms/step
Month-9 - Error: [[7.2259755]]
1/1 [==============================] - 0s 14ms/step
Month-10 - Error: [[3.6851568]]
1/1 [==============================] - 0s 13ms/step
Month-11 - Error: [[2.2634354]]
1/1 [==============================] - 0s 13ms/step
Month-12 - Error: [[13.25774]]
1

In [34]:
display(mae)
display(mape)

10.979341

0.8626157

In [35]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [36]:
errors, mae, mape = year_mae_mape_calculator(trained_model, test_input, test_target, 168)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[83.99913]] - Target[153.199]| =  Error: [[69.199875]]
1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[-14.03441]] - Target[146.76999999999998]| =  Error: [[160.80441]]
1/1 [==============================] - 0s 14ms/step
Ano-11: |Prediction[[-7.420944]] - Target[155.80599999999995]| =  Error: [[163.22694]]


[array([[69.199875]], dtype=float32),
 array([[160.80441]], dtype=float32),
 array([[163.22694]], dtype=float32)]

131.07707

0.8649835